## DeChallenge

Autor: Andres Angarita
Última actualización: Abril 2023

En este notebook se presenta un análisis exploratorio de datos sobre las monedas del mercado utilizando la api de coingecko.

El objetivo de este análisis es identificar ciertos campos del top 120 de las monedas del mercado.

Se utilizó el conjunto de datos markets que se puede obtener con que querystring de la primera seccion.

El análisis se divide en las siguientes secciones:
- Sección 1: Obtencion de datos
- Sección 2: Tratamiento de datos
- Sección 3: Exporte de datos

Nota: Este notebook fue creado utilizando jupyter 1.0.0.


### Seccion 1. Obtencion de datos

In [57]:
import requests
import traceback
import sys
import pandas as pd

try:
    markets = requests.get('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=120&page=1&sparkline=false&locale=en')
    cols=['Id','Name','Current_price','Ath','Ath_date','Market_cap','Market_cap_rank','Circulating_supply','Max_supply','Last_updated','Description_en','Image_large','Links']
    filtered_markets = pd.DataFrame(columns=cols)
    
    if markets.status_code == 200:
        data = markets.json()
        data_dict = {}
        
        for coin in data:
            data_dict['Id'] = coin['id']
            data_dict['Name'] = coin['name']
            data_dict['Current_price'] = coin['current_price']
            data_dict['Ath'] = coin['ath']
            data_dict['Ath_date'] = coin['ath_date']
            data_dict['Market_cap'] = coin['market_cap']
            data_dict['Market_cap_rank'] = coin['market_cap_rank']
            data_dict['Circulating_supply'] = coin['circulating_supply']
            data_dict['Max_supply'] = coin['max_supply']
            data_dict['Last_updated'] = coin['last_updated']
            
            additional_info = requests.get('https://api.coingecko.com/api/v3/coins/'+coin['id'])
            if additional_info.status_code == 200:
                more_info = additional_info.json()
                description_en = more_info["description"]["en"]
                links = more_info["links"]["homepage"]
            
            data_dict['Description_en'] = description_en
            
            img = requests.get(coin['image'], stream=True)
            data_dict['Image_large'] =sys.getsizeof(img.content)
            
            data_dict['Links'] = links[0]
    
            tmpDf = pd.DataFrame.from_dict(data_dict, orient='index').T
            filtered_markets = pd.concat([filtered_markets,tmpDf],ignore_index=True)
    else:
        print('No se pudo obtener los datos de CoinGecko')
except Exception as e:
    print('Error')
    print("Ocurrió un error:", e)
    traceback.print_exc()


### Seccion 2. Tratamiento de datos

In [58]:
from bs4 import BeautifulSoup as bs
from dateutil.parser import parse

def get_exchange(currency="USD", amount=1):
    '''funcion que obtiene el valor del dolar en pesos colombianos 
    Parameters
    ----------
    currency : string
        descripcion de la moneda a convertir, teniendo en cuenta la nomenclatura de la pagina x-rates.com 
    amount : float
        cantidad de moneda a convertir a pesos colombianos
    Returns
    -------
    float 
        valor de la moneda en pesos colombianos
    '''
    
    content = requests.get(f"https://www.x-rates.com/table/?from={currency}&amount={amount}").content
    soup = bs(content, "html.parser")
    # Obtener fecha
    #price_datetime = parse(soup.find_all("span", attrs={"class": "ratesTimestamp"})[1].text)
    exchange_tables = soup.find_all("table")
    exchange_rates = {}
    
    for exchange_table in exchange_tables:
        for tr in exchange_table.find_all("tr"):
            #filas de la tabla
            tds = tr.find_all("td")
            if tds:
                currency = tds[0].text
                exchange_rate = float(tds[1].text)
                exchange_rates[currency] = exchange_rate  
    return exchange_rates["Colombian Peso"]


In [59]:
print("Antes de la conversion de dolares a pesos colombianos de los campos Current_price, Ath y Market_cap")
#Primeros 5 registros
display(filtered_markets.head(5))

##Paso de dolares a pesos colombianos
filtered_markets['Current_price'] = filtered_markets['Current_price'].apply(lambda x: get_exchange(amount=x))
filtered_markets['Ath'] = filtered_markets['Ath'].apply(lambda x: get_exchange(amount=x))
filtered_markets['Market_cap'] = filtered_markets['Market_cap'].apply(lambda x: get_exchange(amount=x))

print("Luego de la conversion de dolares a pesos colombianos de los campos Current_price, Ath y Market_cap")

#revision de los datos
filtered_markets.head(5)

Antes de la conversion de dolares a pesos colombianos de los campos Current_price, Ath y Market_cap


,Id,Name,Current_price,Ath,Ath_date,Market_cap,Market_cap_rank,Circulating_supply,Max_supply,Last_updated,Description_en,Image_large,Links
0,bitcoin,Bitcoin,27474,69045,2021-11-10T14:24:11.849Z,531824236547,1,19355556.0,21000000.0,2023-04-25T00:46:27.201Z,Bitcoin is the first successful internet money...,9441,http://www.bitcoin.org
1,ethereum,Ethereum,1841.84,4878.26,2021-11-10T14:24:19.604Z,221798893848,2,120409031.538542,None,2023-04-25T00:46:35.244Z,"Ethereum is a global, open-source platform for...",10010,https://www.ethereum.org/
2,tether,Tether,1.001,1.32,2018-07-24T00:00:00.000Z,81602572855,3,81538750252.410507,None,2023-04-25T00:45:00.308Z,Tether (USDT) is a cryptocurrency with a value...,10004,https://tether.to/
3,binancecoin,BNB,331.22,686.31,2021-05-10T07:24:17.097Z,52294887191,4,157886280.0,200000000.0,2023-04-25T00:46:33.487Z,Binance Coin is the cryptocurrency of the <a h...,17366,https://www.binance.com/
4,usd-coin,USD Coin,0.999223,1.17,2019-05-08T00:40:28.300Z,30718965511,5,30748502565.5681,None,2023-04-25T00:46:26.552Z,USDC is a fully collateralized US dollar stabl...,13168,https://www.circle.com/en/usdc


Luego de la conversion de dolares a pesos colombianos de los campos Current_price, Ath y Market_cap


,Id,Name,Current_price,Ath,Ath_date,Market_cap,Market_cap_rank,Circulating_supply,Max_supply,Last_updated,Description_en,Image_large,Links
0,bitcoin,Bitcoin,1.224379e+08,3.077592e+08,2021-11-10T14:24:11.849Z,2.370013e+15,1,19355556.0,21000000.0,2023-04-25T00:46:27.201Z,Bitcoin is the first successful internet money...,9441,http://www.bitcoin.org
1,ethereum,Ethereum,8.208161e+06,2.174422e+07,2021-11-10T14:24:19.604Z,9.884211e+14,2,120409031.538542,None,2023-04-25T00:46:35.244Z,"Ethereum is a global, open-source platform for...",10010,https://www.ethereum.org/
2,tether,Tether,4.460957e+03,5.883730e+03,2018-07-24T00:00:00.000Z,3.636524e+14,3,81538750252.410507,None,2023-04-25T00:45:00.308Z,Tether (USDT) is a cryptocurrency with a value...,10004,https://tether.to/
3,binancecoin,BNB,1.476082e+06,3.059138e+06,2021-05-10T07:24:17.097Z,2.330461e+14,4,157886280.0,200000000.0,2023-04-25T00:46:33.487Z,Binance Coin is the cryptocurrency of the <a h...,17366,https://www.binance.com/
4,usd-coin,USD Coin,4.453038e+03,5.215124e+03,2019-05-08T00:40:28.300Z,1.368955e+14,5,30748502565.5681,None,2023-04-25T00:46:26.552Z,USDC is a fully collateralized US dollar stabl...,13168,https://www.circle.com/en/usdc


In [73]:
from datetime import datetime

##Adicion de columnas nuevas
filtered_markets['Days_since_ath'] = (pd.to_datetime('today') - pd.to_datetime(filtered_markets['Ath_date'], format='%Y-%m-%dT%H:%M:%S.%fZ')).dt.days

#Precio ATH en pesos colombianos
filtered_markets['Ath_COP_Price'] = filtered_markets['Ath']

#revision de los datos
filtered_markets.head(5)

,Id,Name,Current_price,Ath,Ath_date,Market_cap,Market_cap_rank,Circulating_supply,Max_supply,Last_updated,Description_en,Image_large,Links,Days_since_ath,Ath_COP_Price
0,bitcoin,Bitcoin,1.224379e+08,3.077592e+08,2021-11-10T14:24:11.849Z,2.370013e+15,1,19355556.0,21000000.0,2023-04-25T00:46:27.201Z,Bitcoin is the first successful internet money...,9441,http://www.bitcoin.org,530,3.077592e+08
1,ethereum,Ethereum,8.208161e+06,2.174422e+07,2021-11-10T14:24:19.604Z,9.884211e+14,2,120409031.538542,None,2023-04-25T00:46:35.244Z,"Ethereum is a global, open-source platform for...",10010,https://www.ethereum.org/,530,2.174422e+07
2,tether,Tether,4.460957e+03,5.883730e+03,2018-07-24T00:00:00.000Z,3.636524e+14,3,81538750252.410507,None,2023-04-25T00:45:00.308Z,Tether (USDT) is a cryptocurrency with a value...,10004,https://tether.to/,1735,5.883730e+03
3,binancecoin,BNB,1.476082e+06,3.059138e+06,2021-05-10T07:24:17.097Z,2.330461e+14,4,157886280.0,200000000.0,2023-04-25T00:46:33.487Z,Binance Coin is the cryptocurrency of the <a h...,17366,https://www.binance.com/,714,3.059138e+06
4,usd-coin,USD Coin,4.453038e+03,5.215124e+03,2019-05-08T00:40:28.300Z,1.368955e+14,5,30748502565.5681,None,2023-04-25T00:46:26.552Z,USDC is a fully collateralized US dollar stabl...,13168,https://www.circle.com/en/usdc,1447,5.215124e+03


### Seccion 3. Exporte de datos



In [74]:
filtered_markets.to_csv('test_challege.csv', index=False)